## 06 R Squared (R^2)

#### RMSE 和 MAE 的问题
* 分类问题可以比较：不同的问题，结果都在0~1这个区间
* 回归问题：比如预测房价误差是1万元；预测成绩，误差是10分，哪个模型更准确呢？

所以我们要统一这个比较水平

![IMAGE](https://farm2.staticflickr.com/1734/42632427042_6ebd7df76d_o.png)
![IMAGE](https://farm2.staticflickr.com/1732/27812357077_cf551d4e43_o.png)

分母是一个比较「粗犷」的模型，分子是我们「比较精密」的模型
![IMAGE](https://farm2.staticflickr.com/1728/27812468387_3ec4f989b5_o.png)

> 它反映了我们的误差和平均值作为误差的相似程度，相似性越高，说明还不如用均值，模型越差。

* R^2 <= 1

* R^2 越大越好。当我们的预测模型不犯任何错误是，R^2 得到最大值 1
> 若果 R^2 = 0，说明我们的模型和平均值作为的模型一样，所以太差了

* 当我们的模型等于基准模型时，R^2 为 0 如果 R^2 <0，说明我们学习到的模型还不如基准模型。**此时，很有可能我们的数据不存在任何线性关系。**

![IMAGE](https://farm2.staticflickr.com/1743/41962986584_d719bb469e_o.png)更好进行运算

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
boston = datasets.load_boston()
x = boston.data[:,5] # 只使用房间数量这个特征
y = boston.target

x = x[y < 50.0]
y = y[y < 50.0]

In [3]:
from playML.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, seed=666)

In [4]:
from playML.SimpleLinearRegression import SimpleLinearRegression

reg = SimpleLinearRegression()
reg.fit(x_train, y_train)

SimpleLinearRegression()

In [5]:
reg.a_

7.8608543562689563

In [6]:
reg.b_

-27.45934280670555

In [7]:
y_predict = reg.predict(x_test)

### R Square

In [8]:
from playML.metrics import mean_squared_error

1 - mean_squared_error(y_test, y_predict)/np.var(y_test)

0.61293168039373236

### 封装我们自己的 R Score

代码参见 [这里](playML/metrics.py) 

In [9]:
from playML.metrics import r2_score

r2_score(y_test, y_predict)

0.61293168039373236

### scikit-learn中的 r2_score

In [10]:
from sklearn.metrics import r2_score

r2_score(y_test, y_predict)

0.61293168039373236

scikit-learn中的LinearRegression中的score返回r2_score:[http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

### 在我们的SimpleRegression中添加score

In [11]:
reg.score(x_test, y_test)

0.61293168039373236

# 关于可决系数R-Square


首先定义一些概念：

样本数据集： ![（x_1,y_1）,（x_2,y_2）,\dots,（x_n,y_n）](https://www.zhihu.com/equation?tex=%EF%BC%88x_1%2Cy_1%EF%BC%89%2C%EF%BC%88x_2%2Cy_2%EF%BC%89%2C%5Cdots%2C%EF%BC%88x_n%2Cy_n%EF%BC%89)

经模型计算得到的预测值: ![\hat{y_1},\hat{y_2},\dots,\hat{y_n}](https://www.zhihu.com/equation?tex=%5Chat%7By_1%7D%2C%5Chat%7By_2%7D%2C%5Cdots%2C%5Chat%7By_n%7D)

- 观测数据均值： ![\bar{y}=\frac{1}{n}\sum\limits_{i=1}^ny_i](https://www.zhihu.com/equation?tex=%5Cbar%7By%7D%3D%5Cfrac%7B1%7D%7Bn%7D%5Csum%5Climits_%7Bi%3D1%7D%5Eny_i)
- **残差**（residual）(与方差成比例)： ![e_i=y_i-\hat{y_i}](https://www.zhihu.com/equation?tex=e_i%3Dy_i-%5Chat%7By_i%7D)
- **总平方和**（total sum of squares）： ![SS_{tot}=\sum_{i}(y_i-\bar{y})^2](https://www.zhihu.com/equation?tex=SS_%7Btot%7D%3D%5Csum_%7Bi%7D%28y_i-%5Cbar%7By%7D%29%5E2)
- **回归平方和，又称可解释平方和**（regression/explained sum of squares）: ![SS_{reg}=\sum_{i}(\hat{y_i}-\bar{y})^2](https://www.zhihu.com/equation?tex=SS_%7Breg%7D%3D%5Csum_%7Bi%7D%28%5Chat%7By_i%7D-%5Cbar%7By%7D%29%5E2)
- **残差平方和**（residual sum of squares）: ![SS_{res}=\sum_{i}(y_i-\hat{y_i})^2=\sum_{i}e_i^2](https://www.zhihu.com/equation?tex=SS_%7Bres%7D%3D%5Csum_%7Bi%7D%28y_i-%5Chat%7By_i%7D%29%5E2%3D%5Csum_%7Bi%7De_i%5E2)

知晓以上概念后，可决系数（coefficient of determination）的定义如下：

![R^2=1-\frac{SS_{res}}{SS_{tot}}](https://www.zhihu.com/equation?tex=R%5E2%3D1-%5Cfrac%7BSS_%7Bres%7D%7D%7BSS_%7Btot%7D%7D)

![R^2](https://www.zhihu.com/equation?tex=R%5E2)为回归平方和与总平方和的比值，反映了自变量对因变量的可解释比例。

例如 ![R^2=0.8](https://www.zhihu.com/equation?tex=R%5E2%3D0.8) ，表示自变量能够解释80%关于因变量的变化。